In [1]:
# mysql
import mysql.connector
# pandas
import pandas as pd

In [2]:
otrs_user = 'dbadminotrs'
otrs_pass = 'jacsos-tygwuv-Dipky6'

cnx = mysql.connector.connect(user=otrs_user, password=otrs_pass,
                              host='otrs.cluster-ro-cwqmr5tsjjh9.us-west-2.rds.amazonaws.com',
                              database='tronex.otrs')

query = ('SELECT T.title, S.* \
         FROM `tronex.otrs`.itv_sabana_tickets AS S \
         INNER JOIN `tronex.otrs`.ticket AS T \
             ON S.ticket_id = T.id \
         WHERE cola = "SERVICIOS IT" AND \
             correo_cliente NOT IN \
                ("soporte@sasaconsultoria.com", \
                "telefonia@tronex.com", \
                 "serviciosit@tronex.com") \
            AND T.title not like "%has low free space" \
            AND agente in ("JAIME ALBERTO GONZALEZ", "ROBINSON CASTRO", "RODOLFO LEON VELEZ") \
            AND estado_ticket <> "merged"')

df = pd.read_sql(query, con=cnx)

years = pd.to_datetime(df['fecha_creacion']).dt.year.unique()
year_list = [{'label': x, 'value': x} for x in years]

In [4]:
df.columns

Index(['title', 'ticket_id', 'ticket_numero', 'estado_ticket', 'tipo_estado',
       'tipo_ticket', 'servicio', 'servicio_padre', 'servicio_hijo', 'cola',
       'sla', 'cumple_sla', 'via_de_ingreso', 'primer_contacto', 'reincidente',
       'agente', 'tipo_agente', 'correo_cliente', 'nombre_usuario',
       'nombre_area', 'centro_costo', 'unidad_negocio', 'empresa', 'merged',
       'nuevo_asignado', 'asignado_en_progreso', 'asignado_pendiente',
       'asignado_cerrado', 'asignado_espera', 'nuevo_cerrado',
       'fecha_creacion', 'fecha_cierre', 'anio', 'mes', 'day',
       'clasificacion_de_ticket', '_sin_SCUN', 'ComTipo', 'Clasificacion'],
      dtype='object')

In [14]:
month_list = pd.to_datetime(df[pd.to_datetime(
        df['fecha_creacion']).dt.year == int(2022)]['fecha_creacion']).dt.month.unique()
month_list

array([2, 1], dtype=int64)

In [7]:
df.loc[(df['day'] == "1") & (df['mes'] == "2")]

,title,ticket_id,ticket_numero,estado_ticket,tipo_estado,tipo_ticket,servicio,servicio_padre,servicio_hijo,cola,...,nuevo_cerrado,fecha_creacion,fecha_cierre,anio,mes,day,clasificacion_de_ticket,_sin_SCUN,ComTipo,Clasificacion


In [12]:
df['mes'].unique()

array(['February', 'January', 'December', 'November', 'October',
       'September', 'August', 'July', 'June', 'May', 'April', 'March'],
      dtype=object)

In [21]:
df['fecha_creacion'] = pd.to_datetime(df['fecha_creacion'])

In [22]:
df.set_index('fecha_creacion', inplace=True)

In [25]:
tmp = df.loc['2022-02']
tmp.sample()

,title,ticket_id,ticket_numero,estado_ticket,tipo_estado,tipo_ticket,servicio,servicio_padre,servicio_hijo,cola,...,asignado_espera,nuevo_cerrado,fecha_cierre,anio,mes,day,clasificacion_de_ticket,_sin_SCUN,ComTipo,Clasificacion
fecha_creacion,,,,,,,,,,,,,,,,,,,,,
2022-02-02,No. 3102657423 sin datos Medellin Android HH,93028,2022020206000075,CERRADO EXITOSAMENTE,CERRADO,REQUERIMENTO/SOLICITUD,AUTOMATIZACION FUERZA DE VENTAS::ADMINISTRACION,AUTOMATIZACION FUERZA DE VENTAS,ADMINISTRACION,SERVICIOS IT,...,0,11:10:32,2022-02-03,2022,February,2,COTIZACION,CONSUMO MASIVO,_SIN COMTIPO,_SIN CLASIFICACION


In [30]:
temp = pd.DataFrame(tmp['servicio'].value_counts()).reset_index().rename(columns={'index': 'servicio', 'servicio': 'cantidad'})

In [31]:
import plotly.express as px

In [35]:
px.bar(
    temp,
    x='servicio',
    y='cantidad',
).update_layout(
    font = {
        'size': 7,
    }
)